In [ ]:
import sys, pickle, os, json, re, time, random, logging, pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns, scipy, sklearn, networkx as nx, importlib

# Read the Excel file
df = pd.read_excel('RQ1 Evaluation.xlsx')

def analyze_metrics_from_columns(df):
    """
    Automatically identify columns with '_Precision' and '_Recall',
    compute precision and recall for each row,
    then calculate the average across all rows.
    """
    # Clean column names by removing '\n'
    df.columns = [col.strip().replace('\n', '') for col in df.columns]
    
    # Identify all model names (based on "_Precision" and "_Recall" columns)
    precision_cols = [col for col in df.columns if '_Precision' in col]
    recall_cols = [col for col in df.columns if '_Recall' in col]
    
    models = set()
    for col in precision_cols + recall_cols:
        model_name = col.split('_')[0]
        if model_name:
            models.add(model_name)
    
    print(f"Detected models: {', '.join(models)}")
    print(f"Precision columns: {', '.join(precision_cols)}")
    print(f"Recall columns: {', '.join(recall_cols)}")
    
    results = {}
    
    for model in models:
        precision_col = f'{model}_Precision'
        recall_col = f'{model}_Recall'
        
        if precision_col not in df.columns:
            print(f"Warning: Precision column for {model} not found")
            continue
        if recall_col not in df.columns:
            print(f"Warning: Recall column for {model} not found")
            continue
            
        precision_values = []
        recall_values = []
        f1_values = []
        
        total_tp = 0
        total_fp = 0
        total_found = 0
        total_miss = 0
        
        for idx, row in df.iterrows():
            if pd.notna(row[precision_col]) and isinstance(row[precision_col], str):
                cell_lower = row[precision_col].lower()
                tp = cell_lower.count('#tp')
                fp = cell_lower.count('#fp')
                total_tp += tp
                total_fp += fp
                if tp + fp > 0:
                    precision_values.append(tp / (tp + fp))
            
            if pd.notna(row[recall_col]) and isinstance(row[recall_col], str):
                cell_lower = row[recall_col].lower()
                found = cell_lower.count('#found')
                miss = cell_lower.count('#miss')
                total_found += found
                total_miss += miss
                if found + miss > 0:
                    recall_values.append(found / (found + miss))
        
        avg_precision = np.mean(precision_values) if precision_values else 0
        avg_recall = np.mean(recall_values) if recall_values else 0
        
        for p, r in zip(precision_values, recall_values):
            if p + r > 0:
                f1_values.append(2 * (p * r) / (p + r))
        
        avg_f1 = np.mean(f1_values) if f1_values else 0
        
        results[model] = {
            'TP': total_tp,
            'FP': total_fp,
            'Avg_Precision': avg_precision,
            'Found': total_found,
            'Miss': total_miss,
            'Avg_Recall': avg_recall,
            'Avg_F1': avg_f1,
            'Row_Count': len(precision_values)
        }
    
    return pd.DataFrame(results).T

# Run the analysis
results_df = analyze_metrics_from_columns(df)
print("\nPrecision and Recall statistics for each model (averaged by rows):")
print(results_df)

# Visualization
plt.figure(figsize=(15, 5))

# Precision comparison
plt.subplot(1, 3, 1)
plt.bar(results_df.index, results_df['Avg_Precision'], color='skyblue')
plt.title('Average Precision per Model')
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.ylabel('Average Precision')

# Recall comparison
plt.subplot(1, 3, 2)
plt.bar(results_df.index, results_df['Avg_Recall'], color='lightgreen')
plt.title('Average Recall per Model')
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.ylabel('Average Recall')

# F1 score comparison
plt.subplot(1, 3, 3)
plt.bar(results_df.index, results_df['Avg_F1'], color='salmon')
plt.title('Average F1 Score per Model')
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.ylabel('Average F1 Score')

plt.tight_layout()
plt.show()

def plot_metrics_by_token_length(df, bin_size=250, fig_width=10, fig_height=10, font_size=10):
    """
    Plot precision and recall curves across different token lengths
    """
    token_col = None
    for col in df.columns:
        if 'token' in col.lower() and 'length' in col.lower():
            token_col = col
            break

    if token_col is None:
        print("Error: Token length column not found!")
        return

    print(f"Using '{token_col}' as the token length data...")

    methods = ["LLM4CTI", "CTIKG", "GPT4o", "extractor"]
    rename_map = {
        "LLM4CTI": "LLM4CTI",
        "CTIKG": "CTIKG",
        "GPT4o": "GPT4o",
        "extractor": "Extractor"
    }

    df = df.dropna(subset=[token_col])
    df[token_col] = pd.to_numeric(df[token_col], errors='coerce')
    df = df.dropna(subset=[token_col])

    def bin_label(length):
        if length > 2000:
            return "2000+"
        else:
            return str(int((length // bin_size) * bin_size + (bin_size / 2)))

    df['token_bin_label'] = df[token_col].apply(bin_label)

    plt.rcParams['font.size'] = font_size

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(fig_width, fig_height))
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

    precision_handles = []
    precision_labels = []
    recall_handles = []
    recall_labels = []
    
    precision_data = {}
    recall_data = {}

    for i, method in enumerate(methods):
        p_col = method + "_Precision"
        r_col = method + "_Recall"

        if p_col not in df.columns or r_col not in df.columns:
            print(f"Warning: Missing {p_col} or {r_col}, skipping {method}")
            continue

        bin_precision = {}
        bin_recall = {}
        bin_samples = {}

        for bin_val, bin_df in df.groupby('token_bin_label'):
            tp = fp = found = miss = 0

            for _, row in bin_df.iterrows():
                if pd.notna(row.get(p_col)) and isinstance(row[p_col], str):
                    cell = row[p_col].lower()
                    tp += cell.count('#tp')
                    fp += cell.count('#fp')

                if pd.notna(row.get(r_col)) and isinstance(row[r_col], str):
                    cell = row[r_col].lower()
                    found += cell.count('#found')
                    miss += cell.count('#miss')

            if tp + fp > 0:
                bin_precision[bin_val] = tp / (tp + fp)

            if found + miss > 0:
                bin_recall[bin_val] = found / (found + miss)

            bin_samples[bin_val] = bin_df.shape[0]

        def sort_key(x):
            return float(x.replace('+', '')) if '+' not in x else 9999

        p_bins = sorted(bin_precision.keys(), key=sort_key)
        p_values = [bin_precision[b] for b in p_bins]

        r_bins = sorted(bin_recall.keys(), key=sort_key)
        r_values = [bin_recall[b] for b in r_bins]

        display_name = rename_map.get(method, method)
        print(f"\nMethod: {display_name}")
        print(f"  Precision bins: {len(p_bins)}, total samples: {sum([bin_samples.get(b, 0) for b in p_bins])}")
        print(f"  Recall bins: {len(r_bins)}, total samples: {sum([bin_samples.get(b, 0) for b in r_bins])}")

        precision_data[display_name] = {bin_val: bin_precision.get(bin_val, np.nan) for bin_val in p_bins}
        recall_data[display_name] = {bin_val: bin_recall.get(bin_val, np.nan) for bin_val in r_bins}

        if p_bins:
            line, = ax1.plot(p_bins, p_values, '-', color=colors[i % len(colors)], label=display_name, alpha=0.8)
            ax1.scatter(p_bins, p_values, color=colors[i % len(colors)], s=30, alpha=0.7)
            precision_handles.append(line)
            precision_labels.append(display_name)

        if r_bins:
            line, = ax2.plot(r_bins, r_values, '-', color=colors[i % len(colors)], label=display_name, alpha=0.8)
            ax2.scatter(r_bins, r_values, color=colors[i % len(colors)], s=30, alpha=0.7)
            recall_handles.append(line)
            recall_labels.append(display_name)

    ax1.set_title(f'Precision by Token Length (Binned by {bin_size} Tokens)')
    ax1.set_xlabel('Token Length')
    ax1.set_ylabel('Precision')
    ax1.grid(True, linestyle='--', alpha=0.7)
    ax1.set_ylim(0, 1.05)
    ax1.legend(precision_handles, precision_labels, loc='best')

    ax2.set_title(f'Recall by Token Length (Binned by {bin_size} Tokens)')
    ax2.set_xlabel('Token Length')
    ax2.set_ylabel('Recall')
    ax2.grid(True, linestyle='--', alpha=0.7)
    ax2.set_ylim(0, 1.05)
    ax2.legend(recall_handles, recall_labels, loc='best')

    plt.tight_layout()
    plt.show()
    
    precision_df = pd.DataFrame(precision_data)
    recall_df = pd.DataFrame(recall_data)
    
    return fig, precision_df, recall_df

# Run plotting
fig, precision_df, recall_df = plot_metrics_by_token_length(df, bin_size=250, fig_width=6, fig_height=10)
fig.savefig('rq1.jpg', dpi=300, bbox_inches='tight')

print("\nPrecision DataFrame:")
print(precision_df)
print("\nRecall DataFrame:")
print(recall_df)

# --- The last part about grouping by "Article Type" ---
methods = ["LLM4CTI", "CTIKG", "GPT4o", "Extractor"]
rename_map = {
    "LLM4CTI": "LLM4CTI",
    "CTIKG": "CTIKG",
    "GPT4o": "GPT4o",
    "Extractor": "Extractor"
}

input_types = sorted(df["Aritcle Type"].drop_duplicates().tolist())

rows = []
for classification, subdf in df.groupby("Aritcle Type", sort=False):
    for method in methods:
        p_col = method + "_Precision"
        r_col = method + "_Recall"
        tp = fp = found = miss = 0
        
        for _, row in subdf.iterrows():
            if pd.notna(row.get(p_col)) and isinstance(row[p_col], str):
                cell = row[p_col].lower()
                tp += cell.count('#tp')
                fp += cell.count('#fp')
            if pd.notna(row.get(r_col)) and isinstance(row[r_col], str):
                cell = row[r_col].lower()
                found += cell.count('#found')
                miss += cell.count('#miss')
        
        precision = tp / (tp + fp) if (tp + fp) else 0
        recall = found / (found + miss) if (found + miss) else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) else 0
        
        rows.append({
            "Input Type": classification,
            "Method": method,
            "Precision": precision * 100,
            "Recall": recall * 100,
            "F1": f1 * 100
        })

df_long = pd.DataFrame(rows)

precision_pivot = df_long.pivot(index="Input Type", columns="Method", values="Precision")
recall_pivot    = df_long.pivot(index="Input Type", columns="Method", values="Recall")
f1_pivot        = df_long.pivot(index="Input Type", columns="Method", values="F1")

df_combined = pd.concat(
    [precision_pivot, recall_pivot, f1_pivot],
    axis=1,
    keys=["Precision", "Recall", "F1"]
)

df_combined = df_combined.swaplevel(axis=1)

df_combined.rename(columns=rename_map, level=0, inplace=True)

desired_outer = [rename_map[m] for m in methods]
df_combined = df_combined.reindex(desired_outer, axis=1, level=0)

desired_inner = ["Precision", "Recall", "F1"]
df_combined = df_combined.reindex(desired_inner, axis=1, level=1)

df_combined = df_combined.reindex(sorted(df_combined.index))

zero_rows = df_combined.apply(lambda row: (row == 0).all(), axis=1)
if zero_rows.any():
    print(f"Removed rows with all zero values: {df_combined[zero_rows].index.tolist()}")
    df_combined = df_combined[~zero_rows]

df_avg = df_combined.mean(axis=0, numeric_only=True).to_frame().T
df_avg.index = ["Average"]

df_final = pd.concat([df_combined, df_avg], axis=0)

all_indices = df_final.index.tolist()
if "Average" in all_indices:
    all_indices.remove("Average")
all_indices = sorted(all_indices)
all_indices.append("Average")
df_final = df_final.reindex(all_indices)

def format_val(x):
    return f"{x:.2f}%" if pd.notnull(x) else ""

df_final = df_final.applymap(format_val)

df_final
